# Data Engineering 101

## Introduction

One of the many things that data scientists really dislike is seeing missing, incomplete, or messy data, especially when it's filled with errors entered by humans. It makes their lives harder than they should be. Luckily, as the Python community grows, so do its tools to address these issues.

In this article, I will show you some of the steps that I usually take before implementing EDA (exploratory data analysis) and how to ensure your database is ready for the next stage. The world of data engineering is much bigger in terms of the tools being used to implement such analyses, but most data engineers deal with this stuff on a daily basis.

As a data analyst or scientist, remember that the lack of completeness and format errors can skew your results, as they may go undetected until a later stage. If you are into reporting, your aggregates and reports will suffer, especially if you're unable to easily track down bugs that may appear during the data preparation process. Instead of simply dropping columns and taking the clean data, we can take a few minutes to perform some health and completeness checks, which can mitigate a lot of problems. Think of it as a general health check when you visit your general practitioner or family doctor.

## The data

The data that will be presented today is a simulation of a university student database. It includes information from the human resources office, the medical center, and the educational board. Imagine that you are a data scientist working for the president's office who has asked you to create a report about the students in the science faculty who will be candidates for becoming astronauts and to be nominated for NASA.

These students have undergone IQ/theoretical tests and physical exams. You have data from the three aforementioned faculties. The report should include the following information:

 - Student identifier
 - Name and last name of the student
 - Age of the student
 - Educational major
 - Academic seniority (which year the student is in)
 - Gender
 - Blood type
 - Grades of the physical exam
 - Grades of the theoretical/IQ exam

You are supplied with three databases: the medical, theoretical, and physical education exams.


#### <u>A. Medical information:  *med.csv*</u>
~~~~
The medical database only contains passport IDs and some student health information, as students first visit the medical center before registering in the HR system. They provide their passport number, later to be able to identify them or cross-check with the Ministry of Health. Some of the data is then sent to the HR system, where student profiles are created in the database. However, the medical data is not sent to HR due to the processing regulations of the university and the consent of the students. This includes medical private data. Blood types and donor information are useful for emergencies. The HR office is open 24/7, but the medical center is not equipped with full hospitalization equipment. Only special personnel can access health data
~~~~



#### <u>B. Physical education exam:</u> *field_exam*
~~~~
For the astronaut contest, before the physical exercise exam, the physical test experts ask candidate students for their passport IDs and request them to agree to grant access to and process the medical records from the medical center. Additionally, the students are asked to provide their student ID so that their grades can be downloaded to the grading system.
~~~~


#### <u>C. Theoretical exam:</u> *theory_exam.csv*
~~~~
The students will also take theoretical and IQ tests, and the test experts will send grades to a centralized grading system. These grades include student IDs, names, scores, and educational information about the student and their seniority.
~~~~


#### <u>D. Tuition fee transactions:</u> *tuition_fees.csv*
~~~~
The another table is an external table from the student affairs office of the university. Students pay there tuitions on-line. And the university's bank send the transactions fees based on the student ID. The Student's office affair is not allowed to see the personal information of the student. Only the ID to make sure the database is complete. If a student wants into ask about the status of the tuition or any other financial details, they have to supply their student ID at the front desk and they can cross check with the history. This report is going to the president of the university, he want to know the social status and financial situation of the students. We will find out by seeing who is on scholarship, who has loans, and who is paying the fees directly.       
~~~~


#### <u> E. Human Resources:</u> *HR.csv*
~~~~
As a data expert, you are allowed to communicate with the people who provided the data, whether it is for corrections, improvement of the data, or enrichment of the data .As you are the data expert in the president's office, you are allowed to process data only for this specific exercise. You can't access sensitive student data. For example, we have addresses of the students and additional health information, but only what is supplied to you.

However, we can ask them to send us missing pieces of information if needed, to correct or enhance our results.

~~~~

#### <u> F. The simulation</u>:  *main.csv*
~~~~
This is the file where I created and simulated the data for this exercise. There are extra columns where you can experiment with and create your own data scenarios
~~~~


## The code

### Importing libraries

In [439]:
import os
import pandas as pd 
from dateutil import parser as date_parser
import re
import numpy as np
import sys
from fuzzywuzzy import process as fw_process
import datetime


In [440]:
# Get a dictionary of all loaded modules
loaded_modules = sys.modules


# Print the name and version of each loaded module
libraries = ['os','pandas','dateutil','re','numpy','sys','fuzzywuzzy','datetime']
for module_name, module in loaded_modules.items():
    if module_name in libraries:
        if hasattr(module, "__version__"):
            print(f"{module_name}=={module.__version__}")

re==2.2.1
dateutil==2.8.2
numpy==1.26.3
pandas==2.1.1
fuzzywuzzy==0.18.0


### Storing the data into a dictionary

One of the things I like to do as a data scientist when working on proof-of-concept projects is to experiment with code where the data is stored in its original shape. Typically, I extract small chunks of data to play with in a test environment. This allows me to work with data that has a small size and doesn't consume too much memory. Additionally, it speeds up the execution of codes and enables quick A/B tests. All of this without having to rewrite the same dataframe, using it as a benchmark.

In [441]:
# We will store them in a dict 
database = {  }
database['HR']  = pd.read_csv("fixtures/HR.csv")
database['MEDICAL']  = pd.read_csv("fixtures/medical.csv")
database['PE_EXAM']  = pd.read_csv("fixtures/field_exam.csv")
database['THEO_EXAM']  =pd.read_csv("fixtures/theory_exam.csv")
database['FEES']  =pd.read_csv("fixtures/tuition_fees.csv")
database['MAIN']  =pd.read_csv("fixtures/main.csv")




### The text cleaning function

It's here where all the magic starts. Below you will see 2 functions
**text_cleaner**: This function is intended to fix the headers of a database. Let's say you got you data as an excel. A human being has messed up with it, added very weird characters which makes it harder to use and store. 

So it removes all the special characters and replaces them with an underscore or any value need. It makes it lowercase. It also replaces digits with characters that resemble the digit. For example 5 is replaced with an "S"

You can also pass a list of strings and it will replace them with that value before removing all the special characters. Check the function below you'll also see some example:

In [442]:
def text_cleaner(val_str, 
                        pre_replacements = None,
                        digit_replacement = True,
                        character_replacer="_"):
    
    """
    This function will clean the strings and replace them with a standardized
    form.  We will remove punctuation. Large spaces replaced witha a single underscore. 
    Digits will be replace other resembling letters. Finally everything will be lower cased. 

    Parameters
    ----------
    val_str : string
        The string to be cleaned
    custom_replacements: string
        This is usually a list of string you want to replace with an "_" before removing them. 
        
       

    Returns
    -------
    val_str_clean: list of string.
        The final cleaned string
    

    """
    val_str_clean = val_str
    
    if pre_replacements:
        val_str_clean =  val_str_clean.translate({ord(i): character_replacer for i in pre_replacements})

    
    # I got this from yje package "string". The underscore was removed
    punct = '!"#$%&\'()*+,-–./:;<=>?@[\\]^`{|}~_'.replace(character_replacer,"")
    digits = '0123456789'
    
    # Remove all the special characters and the spaces at the begining and end
    val_str_clean = val_str_clean.translate({ord(i): "" for i in punct}).lstrip().rstrip().lower()
    
    
    # Replace spaces with "_"
    val_str_clean = re.sub("\s+",character_replacer,val_str_clean)
    
    # replace digits with special characters
    morphological_letters = {
    '0': 'O',
    '1': 'I',
    '2': 'Z',
    '3': 'E',
    '4': 'A',
    '5': 'S',
    '6': 'G',
    '7': 'T',
    '8': 'B',
    '9': 'g'}
    
    if digit_replacement:        
        # Remove all the special characters and the spaces at the begining and end
        val_str_clean = val_str_clean.translate({ord(i): morphological_letters[i] for i in digits})
        
        
    return val_str_clean 
        

In [443]:
text_cleaner(" s.ud_ents. !65/*/*/3983",pre_replacements = ".")

's_ud_ents__GSEgBE'

In [444]:
# not that strings are iterable so whether you pass ["1","2","3"] or "123" it's the same
text_cleaner(" stud@en#ts. !65/*/*/3983sss",
                 digit_replacement = False,
                 character_replacer='-',
                 pre_replacements = "@" )

'stud-ents-653983sss'

### The dataframe optimizing function

Below is our main cleaning function; *optimize_dataframe*. It will do a few things: 

1. It will convert the pandas dataframe into an optimized data format per columns. I use the built in function called [convert_dtype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html). This function is really useful since it minimizes the memory consumption and convert strings which are completely integer into their natural types. 
2. Tries to figure out if a column is a date or a numeric and tries to parse it.  
3. Stores the datatypes in a summary dictionary with both old and new column naming
4. Uses the *text_cleaner* function to rename the columns and edit the date column format to make it in a format that can be parsed. 
5. Returns a dictionary that has the cleaned dataframe and it's meta data describing the column rename and data types.  
6. It creates unique IDs that can be easily sorted and names by the user

In [445]:
def optimize_dataframe(df, 
                        pre_clean_str='.',
                        date_delimter='-',
                        id_prefix="",
                        id_suffix=""):
    """
    This function cleans a database to it's optimal datatype and tries to
    parse dates. You can retrun the metadata or simply the clean dataframe. 

    Parameters
    ----------
    df : Pandas DataFrame
        The dataframe to be cleaned.
    pre_clean_str : str, optional
        A string that contains different character. If you want to do a pre-clean
        to replace specific characters with and "_". The process of the cleaning
        is based on removing all the special characters. 
        Example: "NAME.first" becomes "namefirst", but if you keep the value
        to ".", it becomes "name_first". The default is '.'.

    date_delimter : str, optional
        This value is replaced with the special characters if the columns types
        is detected to be a date. This improves the date parsing. The default is '-'.
    id_prefix : str, optional
        Add a prefix to the ID column values. The default is "".
    id_suffix : TYPE, optional
        Add a suffix to the ID column valies. The default is "".

    Returns
    -------
    dict
        Contains the database and its metadata.
    """

    
    convertion_types = ['bytes',
                     'floating',
                     'integer',
                     'mixed-integer',
                     'mixed-integer-float',
                     'decimal',
                     'complex',
                     'categorical',
                     'boolean',
                     'datetime64',
                     'datetime',
                     'date',
                     'timedelta64',
                     'timedelta',
                     'time',
                     'period',
                     'mixed']
    

    df_use = df.copy().convert_dtypes()
    
    
    # Now we will analyze the data     
    database_meta = {'col_name':[],'clean_col_name':[],"col_type":[],"original_col_type":[]} 
    for col_name,col_type in df_use.dtypes.astype(str).to_dict().items():
        
        optimal_type = col_type
        
        if optimal_type =='string':
            
            # Maybe it's mixed of numbers and strings 
            col_temp = pd.Series(df_use[col_name].unique())
            col_temp = col_temp.apply(pd.to_numeric,errors='coerce').fillna(col_temp)
            
            # Extract after convertig numerical
            analysis_type = pd.api.types.infer_dtype(col_temp)
            
            # This activates if the type changes
            if analysis_type in convertion_types: 
                optimal_type = analysis_type
                
            else: 
                try:
                    # Let's parse dates
                    df_use[col_name].dropna().sample(10).\
                            apply(lambda x : text_cleaner(x,
                                                          digit_replacement=False,     
                                                          character_replacer=date_delimter)).apply(date_parser.parse)   
                    optimal_type = 'datetime'
                    
                except: 
                    pass
        
        # store the information
        database_meta['original_col_type'].append(str(df[col_name].dtypes))
        database_meta['col_type'].append(optimal_type)
        database_meta["col_name"].append(col_name)
        database_meta["clean_col_name"].append(text_cleaner(col_name,pre_replacements=pre_clean_str))
    
    # now we will convert the datetypes into date format    
    """If there is format error, we'll be able to detect it here"""
    for col_name,col_type in zip(database_meta['col_name'],database_meta['col_type']):
        if col_type =='datetime':
            df_use[col_name] = df_use[col_name].apply(lambda x : text_cleaner(x,
                                                    digit_replacement=False,     
                                                    character_replacer=date_delimter)).apply(date_parser.parse)  
    
    
            
    # Rename database: As the renaming is the last step duplicate columns will not cause issues.
    df_use = df_use.rename({i:j for i,j in zip(database_meta["col_name"],database_meta["clean_col_name"])},axis=1)        
    
    
    # Add unique id
    if id_prefix: id_prefix = id_prefix+"_"
    if id_suffix: id_prefix = "_"+id_suffix
    
    
    max_num = df_use.shape[0]
    max_digit_size = len(str(max_num))
    number_list = [str(num).zfill(max_digit_size) for num in range(1, max_num+1)]
    df_use['db_ID'] = [f"{id_prefix}{i}{id_suffix}" for i in number_list]
        
    

    return  {"db":df_use, "metadata": database_meta}


In [446]:
# We will iterate over the databases and store it with it's key name
clean_db = {i:optimize_dataframe(j,id_prefix=i) for i,j in database.items()}

temp = pd.DataFrame(clean_db['MAIN']['metadata'])
temp

,col_name,clean_col_name,col_type,original_col_type
0,Enrollment date,enrollment_date,datetime,object
1,Date of Birth,date_of_birth,datetime,object
2,Passport id,passport_id,string,object
3,email,email,string,object
4,Student.ID,student_id,string,object
5,Student.First Name,student_first_name,string,object
6,Student.Last Name,student_last_name,string,object
7,Address – lat,address_lat,Float64,float64
8,Address ! Long,address_long,Float64,float64
9,Morale %,morale,Float64,float64


The previous databases

In [447]:
database['MAIN'].describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Enrollment date,98,73,2015 | OCTOBER | 10,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date of Birth,98,92,03/07/93,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Passport id,98,98,C161A7431,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
email,98,98,LyndiMuamir@sk.edu.pl,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Student.ID,98,98,D34Q23Q41L,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Student.First Name,98,98,Lyndi,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Student.Last Name,98,98,Muamir,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address – lat,94.0,NaN,NaN,NaN,6.378481,51.35186,-89.95953,-33.896191,9.860808,49.918262,84.336175
Address ! Long,94.0,NaN,NaN,NaN,-20.663295,112.55217,-179.7813,-123.575638,-41.255485,83.88495,175.496775
Morale %,98.0,NaN,NaN,NaN,0.511082,0.252594,0.02915,0.314538,0.533621,0.713352,0.970296


Be careful because some issues can arise. For example, take a look at the "major" column, they are now duplicated, or even some columns might be numeric but instead they are actually contains data and requires cleaning. Take a look at the **tution** column ... I kept the typo on purpose to show you even the column names can have problems ( I'm kidding! I discovered it at this phase it but kept it to show you my human error :D). 

So after you detect these issues it would be nice to clean them up.

In [448]:
clean_db['MAIN']['db']['tution_fee']

0      550 USD
1      780 USD
2     1879 USD
3     1215 USD
4      522 USD
        ...   
93    1850 USD
94    1968 USD
95    1364 USD
96     666 USD
97    1649 USD
Name: tution_fee, Length: 98, dtype: string

In [449]:
# Fixing the tuition columns
clean_db['MAIN']['db']['tution_fee'] = clean_db['MAIN']['db']['tution_fee'].map(lambda x: x.split("USD")[0],na_action='ignore').fillna(0).astype(int)

In [450]:
# removing the empty columns
clean_db['MAIN']['db'] = clean_db['MAIN']['db'].dropna(axis=1, how='all')

In [451]:
clean_db['MAIN']['db'].describe(include='all').T.sort_values("unique")

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
gender,98,2,Female,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
organ_donor,98,2,False,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seniortiy_description,98,3,Freshman,39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
student_gpa_letter,98,8,A+,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
blood_type,98,8,O+,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN
major,98,10,Chemical Engineering,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
major,98,10,Chemical Engineering,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cafetria_comments,96,63,"Soup's warm, bread's soft, satisfying.",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
passport_id,98,98,C195F5934,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
email,98,98,FriedlindSnævar@sk.edu.pl,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


You can see that now we converted it into numbers, ooh ... Look, we have 4 missing addresses.

### Analysis of mergers

When humans enter data manually into a simple database (let's say like an excel or website), common errors can include typos, formatting inconsistencies, missing data, incorrect calculations, or accidental deletion of important information. 

It gets even worse when multiple people are editing the same databases simultaneously or seperately, and the results is the having a non-standardazied database (which was the issues we tried to analyze earlier).These errors can lead to inaccurate analyses, flawed decision-making, and inefficiencies in data processing. 

To mitigate these issues, it's essential to implement data validation rules, utilize cell protection features, maintain clear documentation, establish version control protocols, provide training on data entry best practices, and regularly review and audit the data for accuracy.

Below we will do an exercise on to see what are the effects of such errors specifically when doing table joins and how we can implement the best practices.  



#### Sample merger 

Let's take a look at some our data individually and see what happens when we merge some tables

In [452]:
# Let's create some variables to store our tables 
df_hr   = clean_db['HR']['db'].copy()
df_med  = clean_db['MEDICAL']['db'].copy()
df_pe   = clean_db['PE_EXAM']['db'].copy()
df_theo = clean_db['THEO_EXAM']['db'].copy()
df_fee  = clean_db['FEES']['db'].copy()

As we mentioned we have the 3 tables that we need to merge: medical records, physical exam, and the IQ tests. Let's quickly take a snapshots of them to see completeness and find out the primary and foreign keys: 


In [453]:
df_pe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   passport_id       87 non-null     string
 1   field_exam_score  92 non-null     Int64 
 2   student_id        80 non-null     string
 3   db_ID             95 non-null     object
dtypes: Int64(1), object(1), string(2)
memory usage: 3.2+ KB


In [454]:
df_theo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   student_id  98 non-null     string
 1   score       98 non-null     Int64 
 2   db_ID       98 non-null     object
dtypes: Int64(1), object(1), string(1)
memory usage: 2.5+ KB


In [455]:
df_med.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_of_birth       98 non-null     datetime64[ns]
 1   passport_id         98 non-null     string        
 2   student_first_name  98 non-null     string        
 3   student_last_name   98 non-null     string        
 4   blood_type          98 non-null     string        
 5   gender              98 non-null     string        
 6   db_ID               98 non-null     object        
dtypes: datetime64[ns](1), object(1), string(5)
memory usage: 5.5+ KB


Looks like the IQ exam is complete and everyone has taken the test. So we can try to add additional information to it.

In [456]:
# merge student
"""The theo student table is complete and we merge it with the practical"""
df_report = pd.merge(df_theo,
                     df_pe,
                     on='student_id',
                     how='outer',
                     suffixes=("_theo","_pe"),
                    indicator='merger')
df_report.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   student_id        98 non-null     string  
 1   score             98 non-null     Int64   
 2   db_ID_theo        98 non-null     object  
 3   passport_id       87 non-null     string  
 4   field_exam_score  92 non-null     Int64   
 5   db_ID_pe          95 non-null     object  
 6   merger            113 non-null    category
dtypes: Int64(2), category(1), object(2), string(2)
memory usage: 5.9+ KB


In [457]:
df_report.merger.value_counts()

merger
both          80
left_only     18
right_only    15
Name: count, dtype: int64

No duplicates, but it seems a lot of the physical exam records were not merged. Well the only reason could be that the student IDs are missing or they are faulty. Doing an out join tells us if there are any issues with both tables. Left joins would be left for after the investigation process. Let's see what's going on in the right table

In [458]:
df_pe

,passport_id,field_exam_score,student_id,db_ID
0,F193I8987_x,12,O69K36F35N,PE_EXAM_01
1,G213H6876,65,<NA>,PE_EXAM_02
2,E122C7613,66,N95Q83K66L,PE_EXAM_03
3,E236I5047,79,F23F98R36E,PE_EXAM_04
4,I119I9824,94,<NA>,PE_EXAM_05
...,...,...,...,...
90,I161H6029,58,V42V13R33C,PE_EXAM_91
91,Expired passport,11,A40E12X76K,PE_EXAM_92
92,H104H6344,14,F96U81D87W,PE_EXAM_93
93,I212A5244,67,G16E29S55N,PE_EXAM_94


In [459]:
df_report.query("merger =='right_only'")

,student_id,score,db_ID_theo,passport_id,field_exam_score,db_ID_pe,merger
98,<NA>,<NA>,NaN,G213H6876,65,PE_EXAM_02,right_only
99,<NA>,<NA>,NaN,I119I9824,94,PE_EXAM_05,right_only
100,<NA>,<NA>,NaN,D195D9493,24,PE_EXAM_12,right_only
101,<NA>,<NA>,NaN,D142I7007,48,PE_EXAM_19,right_only
102,<NA>,<NA>,NaN,D161I6944,10,PE_EXAM_24,right_only
103,<NA>,<NA>,NaN,G222G8347,76,PE_EXAM_28,right_only
104,<NA>,<NA>,NaN,A158E6453,35,PE_EXAM_34,right_only
105,<NA>,<NA>,NaN,H268D7080,91,PE_EXAM_37,right_only
106,<NA>,<NA>,NaN,C140A6549,15,PE_EXAM_43,right_only
107,<NA>,<NA>,NaN,A107E7669,79,PE_EXAM_50,right_only


In [460]:
df_report.query("merger =='left_only'")

,student_id,score,db_ID_theo,passport_id,field_exam_score,db_ID_pe,merger
1,S67Y79P97N,58,THEO_EXAM_02,<NA>,<NA>,NaN,left_only
4,X84A95B48H,73,THEO_EXAM_05,<NA>,<NA>,NaN,left_only
11,U30F35G12E,61,THEO_EXAM_12,<NA>,<NA>,NaN,left_only
18,A32J48B87E,35,THEO_EXAM_19,<NA>,<NA>,NaN,left_only
23,Y64Z79C83L,65,THEO_EXAM_24,<NA>,<NA>,NaN,left_only
27,S85H92U16R,87,THEO_EXAM_28,<NA>,<NA>,NaN,left_only
30,Y68K44O33E,12,THEO_EXAM_31,<NA>,<NA>,NaN,left_only
31,T23W58G70E,60,THEO_EXAM_32,<NA>,<NA>,NaN,left_only
32,K50V21E10E,86,THEO_EXAM_33,<NA>,<NA>,NaN,left_only
36,K33F51P59O,46,THEO_EXAM_37,<NA>,<NA>,NaN,left_only


#### Investigate the IDS

Let's check if the ids are complete

We can clearly see in the physical exam records that many students did not provide their student IDs, maybe they didn't do the same or there we typos. To improve the merge, we need to address this issue. 

As you have access to the HR database, you can request student IDs and passport IDs from the HR department. Upon calling the HR department, provide them with all the passport IDs and student IDs available and request additional student information such as names and academic levels. Since you already obtained passport consents from the physical exam, everyone is identifiable. In return, the HR department sends you the data.

It's crucial to note that accessing data from other departments is not permissible without proper clearance. The delay in merging the data initially was due to the necessity to prove the absence of data and obtain appropriate permissions. 

In this project even though it is a simulation, if you are working in a highly regulated industry, data security is paramount, and you will face more issues like this. 


What is interesting is that the medical database has passport values which are complete and correct. The HR database has a complete set of data for all the students, this will help us to create a correct identification database.

In [461]:
# merge student 
"""The medical table can have passport ids we don't know about. Plus we have no access to it"""
df_ids= pd.merge(df_hr, df_med,
                  on='passport_id',
                  how='outer',
                  suffixes=("_hr",'_med'))[['passport_id','student_id','gender','student_first_name_hr','student_last_name_hr']]
df_ids.rename({'student_first_name_hr':'first_name','student_last_name_hr':'last_name'},axis=1,inplace=True)

df_ids

,passport_id,student_id,gender,first_name,last_name
0,F193I898722,O69K36F35N,Female,Refat,Natali
1,G213H6876,S67Y79P97N,Male,Arianna,Kim
2,E122C7613,N95Q83K66L,Female,Inge,Bego
3,E236I5047,F23F98R36E,Male,Alush,Bauyrzhan
4,I119I9824,X84A95B48H,Male,Luo+Ping,Palbo
...,...,...,...,...,...
93,I161H6029,V42V13R33C,Female,Trini,Dong+Keun
94,B205A8589,A40E12X76K,Female,Lovise,Ji+Kui
95,H104H6344,F96U81D87W,Female,Ishilde,Jalldëz
96,I212A5244,G16E29S55N,Female,Joselyne,Godi


Below is a function that will check the status of a dirty list of ids compared to a set of correct ones. This will help us to fix the ids of the database in interest.

In [462]:
def id_checker(dirty_list, clean_list, name=None,missing_values=True):
    """
    This function takes 2 lists and compares them. 
    It returns a dictionary which tells you will ids to fix and the ones that are missing. 
    """
    dirty_set = set(dirty_list)
    clean_set = set(clean_list)
    
    fix = dirty_set - clean_set.intersection(dirty_set)
    missing = clean_set - clean_set.intersection(dirty_set)
    if missing_values:
        res = {'fix':fix}
        res = {'fix':fix,'missing':missing}
        if len(fix) == 0 and len(missing)==0: print(f"[INFO]: CLEAN -> {name}")
        if len(fix) != 0: print(f"[INFO]: FIX ({len(fix)})-> {name}")
        if len(missing)!=0: print(f"[INFO]: MISSING ({len(missing)})-> {name}") 
    else: 
        res = {'fix':fix,'missing':missing}
        if len(fix) == 0 and len(missing)==0: print(f"[INFO]: CLEAN -> {name}")
        if len(fix) != 0: print(f"[INFO]: FIX ({len(fix)})-> {name}")

    return res    
        
# For the theoretical
theo_ids = id_checker(df_theo.student_id.dropna(),df_ids.student_id,'IQ exam - student id')
prac_ids = id_checker(df_pe.student_id.dropna(),df_ids.student_id,'Physical exam - student id')
prac_pass = id_checker(df_pe.passport_id.dropna(),df_ids.passport_id,'Physical exam - passports')
fee_ids = id_checker(df_fee.student_id.dropna(),df_ids.student_id,'Fees - student id')
fee_pass = id_checker(df_fee.passport_id.dropna(),df_ids.passport_id,'Fees - passports')

    

[INFO]: CLEAN -> IQ exam - student id
[INFO]: MISSING (18)-> Physical exam - student id
[INFO]: FIX (3)-> Physical exam - passports
[INFO]: MISSING (18)-> Physical exam - passports
[INFO]: MISSING (2)-> Fees - student id
[INFO]: MISSING (2)-> Fees - passports


In [463]:
fee_ids

{'fix': set(), 'missing': {'Z84Y31Z35Q', 'Z97G71Y84S'}}

In [464]:
prac_pass

{'fix': {'<N/A>', 'Expired passport', 'F193I8987_x'},
 'missing': {'A114C7641',
  'A165E9973',
  'A286D8586',
  'B205A8589',
  'C180A9432',
  'C277F5942',
  'D172B8726',
  'D174I8785',
  'D176B7482',
  'E282F5666',
  'F193I898722',
  'F202B6991',
  'F242E6691',
  'G123C6077',
  'G226B9008',
  'G237G5382',
  'G263C6339',
  'G288H6343'}}

Sometimes even different issues can occure. Look at the fees database. It shows that there are no errors. The IDs are correct but we having missing student informations. 

In [465]:
df_fee.describe(include='all')

,transaction_id,student_id,passport_id,tuition_fees,semester,loan,scholarship,db_ID
count,192.0,175,176,192.0,192.0,192,192,192
unique,<NA>,96,96,<NA>,<NA>,2,2,192
top,<NA>,S67Y79P97N,B165I8646,<NA>,<NA>,True,False,FEES_177
freq,<NA>,2,2,<NA>,<NA>,165,184,1
mean,98.177083,NaN,NaN,722.666667,1.5,NaN,NaN,NaN
std,56.826915,NaN,NaN,149.367369,0.501307,NaN,NaN,NaN
min,1.0,NaN,NaN,503.0,1.0,NaN,NaN,NaN
25%,48.75,NaN,NaN,585.0,1.0,NaN,NaN,NaN
50%,98.5,NaN,NaN,702.0,1.5,NaN,NaN,NaN
75%,146.25,NaN,NaN,852.25,2.0,NaN,NaN,NaN


The fees database has multiple transaction information, so we need to aggregate it to get a final clean usable database, where each student will have a summary about themselves. 

In [466]:
summary = df_fee.groupby(["student_id",'passport_id'],dropna=False).agg({'tuition_fees':'sum','db_ID':'count'}).reset_index()
summary.sort_values("db_ID",ascending=False)

,student_id,passport_id,tuition_fees,db_ID
59,N46D16J18G,G236C7367,1470,2
56,N16E91G49J,D114D5052,1449,2
54,M61K98C62H,C277F5942,1218,2
53,M19B87Z20V,D288G5298,1489,2
96,W83C28W43D,A135H7268,1646,2
...,...,...,...,...
41,K33F51P59O,A158E6453,592,1
39,H81O65A56M,<NA>,890,1
38,H81O65A56M,A107E7669,576,1
35,G38B13O78Q,G194I7411,972,1


The database is composed of 2 semesters worth of payments and in some cases students either didn't supply their student ID or the passport. And since the IDs are complete what we can do here is use the database itself to cross enrich between the 2 IDs and as a result we can get a unified aggregated tuition fees table

In [467]:
st_map = {}
pass_map = {}
for st_id, pass_id in zip(summary.student_id,summary.passport_id):
    if pd.isna(st_id) or pd.isna(pass_id): pass
    else: 
        st_map[st_id] = pass_id
        pass_map[pass_id] = st_id 
        

df_fee['student_id_clean'] = df_fee.passport_id.map(pass_map).fillna(df_fee['student_id'])
df_fee['passport_id_clean'] = df_fee.student_id.map(st_map).fillna(df_fee['passport_id'])


**You have to fix for the other columns since the previous dates caused issues for those qualitative ones**

In [468]:
summary = df_fee.groupby(["student_id_clean",'passport_id_clean']).agg({'tuition_fees':'sum','loan':'sum','scholarship':'sum'}).reset_index()
summary.loc[summary.loan==0,'loan'] = 0
summary.loc[summary.loan!=0,'loan'] = 1
summary.loc[summary.scholarship==0,'scholarship'] = 0
summary.loc[summary.scholarship!=0,'scholarship'] = 1

summary.sort_values("scholarship",ascending=False)

,student_id_clean,passport_id_clean,tuition_fees,loan,scholarship
89,Y68K44O33E,A165E9973,1194,0,1
88,Y64Z79C83L,D161I6944,1238,0,1
87,Y49Y11T74M,G288H6343,1648,0,1
86,X84A95B48H,I119I9824,1631,0,1
68,S85H92U16R,G222G8347,1797,1,0
...,...,...,...,...,...
27,G16E29S55N,I212A5244,1616,1,0
26,F96U81D87W,H104H6344,1459,1,0
25,F74B39J14J,C134B8514,1403,1,0
24,F29F90D95Y,H238H6612,1765,1,0


In [469]:
df_fee_clean = summary.copy().rename({"student_id_clean":"student_id","passport_id_clean":'passport_id'},axis=1)

Seems like the guys who have a scholarship are getting fundings from some external organization. Let's see what happened to those 2 missing students from the HR file. 

In [470]:
fee_ids = id_checker(df_fee_clean.student_id.dropna(),df_ids.student_id,'Fees - student id')
fee_ids

[INFO]: MISSING (2)-> Fees - student id


{'fix': set(), 'missing': {'Z84Y31Z35Q', 'Z97G71Y84S'}}

In [471]:
temp = fee_ids['missing']

In [472]:
df_miss = df_hr.query("student_id in @temp")
df_miss[['student_id','passport_id','scholarship']]

,student_id,passport_id,scholarship
60,Z84Y31Z35Q,F212E6749,1
68,Z97G71Y84S,H292E6722,1


Oh look at that! They have scholarships! Must be they have an agreement with the university itself

In [473]:
df_fee_final= pd.concat([df_fee_clean,df_miss[['student_id','passport_id','scholarship']]])

In [474]:
df_fee_final.fillna(0,inplace=True)
df_fee_final

,student_id,passport_id,tuition_fees,loan,scholarship
0,A23J68Q95Z,D176B7482,1739,1,0
1,A26D49U61M,B165I8646,1717,1,0
2,A30W79B50M,H214E7302,1372,1,0
3,A32J48B87E,D142I7007,1074,1,0
4,A38U71K78K,B207I9427,1073,1,0
...,...,...,...,...,...
93,Z24D47P30W,D144I7429,1521,0,0
94,Z75S49G13B,G240A8482,1719,0,0
95,Z77O99Y48B,I224H7974,1892,1,0
60,Z84Y31Z35Q,F212E6749,0,0,1


#### Cleaning the tables fo  

Now that we understand the mechanism, let create a function which will do this enrichment for us


Be carefull that some of the values have a NA nature or some flags which say the need to be removed. 

In [475]:
def fix_ids(df_dirty,
            df_clean, 
            id_col_pairs,
            replacement = ['<N/A>', 'Expired passport']):
    
    # Replace the replacements with NaN
    df_use = df_dirty.copy().replace({i:np.nan for i in replacement})

    for id_col in id_col_pairs:
        dirty_id_list = df_use[id_col].replace({i:np.nan for i in replacement}).dropna().tolist()
        clean_id_list = df_clean[id_col].replace({i:np.nan for i in replacement}).dropna().tolist()

        # 1st check
        check = id_checker(dirty_id_list,clean_id_list,id_col,missing_values=False)


        #Apply a fuzzy match if possible 
        if len(check['fix'])!=0:
            fix_map = {}
            for bad_id in list(check['fix']):
                nearest_string, _ = fw_process.extractOne(str(bad_id), clean_id_list)
                fix_map[bad_id] = nearest_string    

            df_use[id_col].replace(fix_map,inplace=True)   
            
        
        
    #impute by cross matching
    cross_mapping = df_clean.dropna().set_index(id_col_pairs[0])[id_col_pairs[1]].to_dict()
    df_use[id_col_pairs[1]].fillna(df_use[id_col_pairs[0]].map(cross_mapping),inplace=True)
    
    #impute by cross matching
    cross_mapping = df_clean.dropna().set_index(id_col_pairs[1])[id_col_pairs[0]].to_dict()
    df_use[id_col_pairs[0]].fillna(df_use[id_col_pairs[1]].map(cross_mapping),inplace=True)
    

    return df_use
    

In [476]:

df_pe_final = fix_ids(clean_db['PE_EXAM']['db'], df_ids, ['student_id','passport_id'])

[INFO]: FIX (1)-> passport_id


In [477]:
df_pe_final

,passport_id,field_exam_score,student_id,db_ID
0,F193I898722,12,O69K36F35N,PE_EXAM_01
1,G213H6876,65,S67Y79P97N,PE_EXAM_02
2,E122C7613,66,N95Q83K66L,PE_EXAM_03
3,E236I5047,79,F23F98R36E,PE_EXAM_04
4,I119I9824,94,X84A95B48H,PE_EXAM_05
...,...,...,...,...
90,I161H6029,58,V42V13R33C,PE_EXAM_91
91,B205A8589,11,A40E12X76K,PE_EXAM_92
92,H104H6344,14,F96U81D87W,PE_EXAM_93
93,I212A5244,67,G16E29S55N,PE_EXAM_94


In [478]:
df_president = pd.merge(df_pe_final,
                        df_theo,
                         on=['student_id'],
                         how='outer',
                         suffixes=("_pe","_theo")).rename({"field_exam_score":'PE_score','score':'IQ_score'},axis=1).drop(['db_ID_theo','db_ID_pe'],axis=1)

In [479]:
df_president = fix_ids(df_president, df_ids, ['student_id','passport_id'])

[INFO]: CLEAN -> student_id


In [480]:
df_president[['passport_id','student_id','PE_score','IQ_score']]

,passport_id,student_id,PE_score,IQ_score
0,F193I898722,O69K36F35N,12,71
1,G213H6876,S67Y79P97N,65,58
2,E122C7613,N95Q83K66L,66,88
3,E236I5047,F23F98R36E,79,69
4,I119I9824,X84A95B48H,94,73
...,...,...,...,...
93,I212A5244,G16E29S55N,67,74
94,C161A7431,D34Q23Q41L,88,47
95,A165E9973,Y68K44O33E,<NA>,12
96,G123C6077,T23W58G70E,<NA>,60


Let's wrap up the data
- Student identifier -> Done 
- Name and last name of the student -> To be added
- Age of the student -> to be calculated
- Educational major -> to be added 
- Academic seniority (which year the student is in) -> to be added 
- Gender -> to be added 
- Blood type -> To be added 
- Grades of the physical exam -> Done 
- Grades of the theoretical/IQ exam -> Done 

In [481]:
df_hr.columns

Index(['enrollment_date', 'date_of_birth', 'passport_id', 'email',
       'student_id', 'student_first_name', 'student_last_name',
       'seniortiy_description', 'major', 'scholarship', 'db_ID'],
      dtype='object')

In [482]:
df_president = pd.merge(df_president,df_hr,on=['student_id','passport_id'])

In [483]:
df_president = df_president[['passport_id','student_id','student_first_name','student_last_name','date_of_birth','PE_score','IQ_score','major','seniortiy_description']]
df_president

,passport_id,student_id,student_first_name,student_last_name,date_of_birth,PE_score,IQ_score,major,seniortiy_description
0,F193I898722,O69K36F35N,Refat,Natali,1994-08-08,12,71,Civil Engineering,Freshman
1,G213H6876,S67Y79P97N,Arianna,Kim,1994-06-14,65,58,Civil Engineering,Sophmore
2,E122C7613,N95Q83K66L,Inge,Bego,1993-03-05,66,88,Chemical Engineering,Senior
3,E236I5047,F23F98R36E,Alush,Bauyrzhan,1995-11-27,79,69,Computer Science,Sophmore
4,I119I9824,X84A95B48H,Luo+Ping,Palbo,1993-12-28,94,73,Civil Engineering,Sophmore
...,...,...,...,...,...,...,...,...,...
93,I212A5244,G16E29S55N,Joselyne,Godi,1993-06-17,67,74,Biochemistry,Freshman
94,C161A7431,D34Q23Q41L,Lyndi,Muamir,1995-02-16,88,47,Biology,Sophmore
95,A165E9973,Y68K44O33E,Danielle,Fawzi,1995-01-15,<NA>,12,Environmental Science,Freshman
96,G123C6077,T23W58G70E,Stefanino,Chedva,1993-02-01,<NA>,60,Environmental Science,Senior


In [484]:
df_med

,date_of_birth,passport_id,student_first_name,student_last_name,blood_type,gender,db_ID
0,1994-08-08,F193I898722,Refat,Natali,O+,Female,MEDICAL_01
1,1994-06-14,G213H6876,Arianna,Kim,A-,Male,MEDICAL_02
2,1993-03-05,E122C7613,Inge,Bego,O+,Female,MEDICAL_03
3,1995-11-27,E236I5047,Alush,Bauyrzhan,A+,Male,MEDICAL_04
4,1993-12-28,I119I9824,Luo+Ping,Palbo,O-,Male,MEDICAL_05
...,...,...,...,...,...,...,...
93,1995-11-28,I161H6029,Trini,Dong+Keun,AB+,Female,MEDICAL_94
94,1993-03-28,B205A8589,Lovise,Ji+Kui,AB+,Female,MEDICAL_95
95,1995-03-27,H104H6344,Ishilde,Jalldëz,B+,Female,MEDICAL_96
96,1993-06-17,I212A5244,Joselyne,Godi,B-,Female,MEDICAL_97


In [485]:
df_president = pd.merge(df_president,df_med[['passport_id','blood_type','gender']])

In [486]:
current_date = datetime.date.today()
df_president['Age'] = np.round((date_parser.parse(current_date.strftime("%d-%m-%Y"))- df_president['date_of_birth']).dt.days/365)

In [490]:
df_president.sort_values("IQ_score")

,passport_id,student_id,student_first_name,student_last_name,date_of_birth,PE_score,IQ_score,major,seniortiy_description,blood_type,gender,Age
85,E277A9345,U43X69O71Q,Ína,Amédé,1995-10-12,8,2,Mathematics,Freshman,O-,Female,28.0
65,H292E6722,Z97G71Y84S,Shi+Liang,Magnþóra,1994-04-05,64,2,Biochemistry,Sophmore,B-,Male,30.0
68,C236F6011,F24U56Y56Y,Chien+Min,Irfan,1994-12-24,88,6,Biology,Freshman,B-,Female,29.0
60,F237I6757,J85O89G46H,Fachtna,Seo,1994-01-22,23,6,Mechanical Engineering,Freshman,AB+,Female,30.0
36,H268D7080,K42B47E15J,Su+Chen,Elve,1993-09-06,91,7,Electrical Engineering,Freshman,O+,Male,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9,C195F5934,N95K98P24K,Friedlind,Snævar,1994-07-16,80,92,Environmental Science,Sophmore,AB-,Male,30.0
45,E222C5387,C41L29D24E,Maksym,Alexej,1993-02-16,25,94,Biochemistry,Freshman,A-,Female,31.0
22,I229G8627,H52N33P76Z,Ihno,Ekke,1993-08-17,78,97,Physics,Freshman,AB-,Male,31.0
57,F212E6749,Z84Y31Z35Q,Eunice,So Hwan,1993-03-07,9,99,Mathematics,Sophmore,B-,Male,31.0


In [495]:
df_president = pd.merge(df_president,df_fee_final,on='student_id')

In [497]:
df_president

,passport_id,student_id,student_first_name,student_last_name,date_of_birth,PE_score,IQ_score,major,seniortiy_description,blood_type,gender,Age,tuition_fees,loan,scholarship
0,F193I898722,O69K36F35N,Refat,Natali,1994-08-08,12,71,Civil Engineering,Freshman,O+,Female,30.0,1522,1,0
1,G213H6876,S67Y79P97N,Arianna,Kim,1994-06-14,65,58,Civil Engineering,Sophmore,A-,Male,30.0,1468,1,0
2,E122C7613,N95Q83K66L,Inge,Bego,1993-03-05,66,88,Chemical Engineering,Senior,O+,Female,31.0,1626,1,0
3,E236I5047,F23F98R36E,Alush,Bauyrzhan,1995-11-27,79,69,Computer Science,Sophmore,A+,Male,28.0,1779,1,0
4,I119I9824,X84A95B48H,Luo+Ping,Palbo,1993-12-28,94,73,Civil Engineering,Sophmore,O-,Male,30.0,1631,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,I212A5244,G16E29S55N,Joselyne,Godi,1993-06-17,67,74,Biochemistry,Freshman,B-,Female,31.0,1616,1,0
94,C161A7431,D34Q23Q41L,Lyndi,Muamir,1995-02-16,88,47,Biology,Sophmore,O+,Female,29.0,1690,1,0
95,A165E9973,Y68K44O33E,Danielle,Fawzi,1995-01-15,<NA>,12,Environmental Science,Freshman,B-,Male,29.0,1194,0,1
96,G123C6077,T23W58G70E,Stefanino,Chedva,1993-02-01,<NA>,60,Environmental Science,Senior,AB+,Male,31.0,1433,1,0


Now the data science starts ... From garbage to reusable raw data 

## Conclusion

We have taken a simulation of a university data and worked to understand its componenets, we analyzed and made sure the data is complete and imputed the data based on existing data from other tables where the values are similar and connected. Without any predictive analytics or assumptions.

The things we did included:
1. Standardizing the data
2. Optimizing the format and data types
3. Cleaning the databases
4. Imputing missing ID
5. Merging tables

All of this is part of a data engieer's arsenal, such as querying relational databases and setting up a pipeline which ensures the other data teams and domain experts will be able to use the data in an efficient way, knowing the databases are complete and usebale to help them make descisions and predict what they need. 